To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [27]:
from pynq import Overlay
ol = Overlay("hardware/design_qspi.bit")

To run only if you made change in the overlay :

In [28]:
ol.download()
# ol?

We define our SPI hardware object : AxiQspi.

In [29]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

Open the config file

In [30]:
with open("style.css","r") as file:
        css_file = file.read()
# print(css_file)

In [46]:
import yaml
with open("config.yaml","r") as file:
        config_data = yaml.safe_load(file)
        # print(config_data)

In [47]:
%run nb/messages.ipynb

# Config setup
global activate_exceptions # No mandatory using jupyterlab but will be if using a new method
activate_exceptions = config_data["LAUNCH_CONFIG"]["activate_exceptions"]

single_test_config = config_data["LAUNCH_CONFIG"]["single_test"]
all_test_config = config_data["LAUNCH_CONFIG"]["all_tests"]

if (single_test_config["bool"] and all_test_config["bool"]):
    test_config_error()

Define the result file : don't forget to change its name in config.yaml if you don't want to erase the last results

In [48]:
result_file_path = "out/" + config_data["RESULT_FILE"]["name"] + '_results.txt'

In [49]:
%run nb/parse.ipynb
%run nb/translator.ipynb
%run nb/spi/spi_driver.ipynb
%run nb/spi/spi_pkg.ipynb
%run nb/messages.ipynb
%run nb/converter.ipynb
%run nb/package.ipynb


import os 
#change to cnfg_master_spi
cnfg_master_spi(AxiQspi, clk_phase=0, clk_pol=0)
# ALL tests config :
if (all_test_config["bool"]) :
    all_test_files = os.listdir("./data")
    ignored_test_files = all_test_config['ignored_test_files']
    list_test_files = [f for f in all_test_files if f not in ignored_test_files]
    for test_file in list_test_files:
        test_file = test_file[:-4] # remove the txt extension
        result_file_path = "out/" + test_file + '_results.html'
        test_file_path = 'data/' + test_file + '.txt'
        print(f"Currently sending test data from {test_file + '.txt'} ")

        rd_data_queue = [] 
        with open(result_file_path,"w") as result_file:
            result_file_title = "Output Data in an HTML file"
            header_content = f'Results from {test_file}'
            html_content = initiate_html_content(result_file_title, css_file, header_content)
            result_file.write(html_content)

            with open(test_file_path,"r") as test_file_opened:
                parsed_file = run_parse_loop(test_file_opened, result_file)
                for parsed_line in parsed_file:
                    command_disjunction(AxiQspi, result_file, parsed_line, rd_data_queue)

            result_file.write(end_html())
        print(f"Finished sending test data from {test_file + '.txt'} ")
     
    
    
# SINGLE test config :
if (single_test_config["bool"]) :
    test_file = single_test_config['test_file'] 
    result_file_path = "out/" + test_file + '_results.html'
    test_file_path = 'data/' + test_file + '.txt'
    print(f"Currently sending test data from {test_file + '.txt'} ")

    rd_data_queue = [] 
    with open(result_file_path,"w") as result_file:
        header_content = f'Results from {test_file}'
        html_content = initiate_html_content(result_file_title, css_file, header_content)
        result_file.write(html_content)

        with open(test_file_path,"r") as test_file_opened:
            parsed_file = run_parse_loop(test_file_opened, result_file)
            for parsed_line in parsed_file:
                command_disjunction(AxiQspi, result_file, parsed_line, rd_data_queue)

        result_file.write(end_html())
    print(f"Finished sending test data from {test_file + '.txt'} ")

Configure device
Currently sending test data from test_cmp.txt 
Finished sending test data from test_cmp.txt 
Currently sending test data from test2.txt 
Finished sending test data from test2.txt 
Currently sending test data from test1.txt 
Finished sending test data from test1.txt 
Currently sending test data from test_bist_basic.txt 
Finished sending test data from test_bist_basic.txt 
Currently sending test data from test1_onchip_clk.txt 
Finished sending test data from test1_onchip_clk.txt 
Currently sending test data from test_bist_halt_example.txt 
Finished sending test data from test_bist_halt_example.txt 
Currently sending test data from test3.txt 
Finished sending test data from test3.txt 
Currently sending test data from test_bist_array_0_full_range.txt 
Finished sending test data from test_bist_array_0_full_range.txt 
Currently sending test data from test4.txt 
Finished sending test data from test4.txt 
